### 6.8. Stuff LCEL Chain

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human"), "{question}"
])

chain = {
    "context": retriever,  # retriever: List[Doc]
    "question": RunnablePassthrough(),
} | prompt | llm

# The question will be passed to RunnablePassthrough
chain.invoke("Describe Victory Mansions")


[nltk_data] Downloading package punkt to /Users/I538638/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/I538638/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


AIMessage(content='Victory Mansions is a building where Winston Smith lives. It is described as having glass doors that let in gritty dust, a hallway that smells of boiled cabbage and old rag mats, and a poster of a large face with the caption "BIG BROTHER IS WATCHING YOU." The building has a faulty lift, and Winston has to climb seven flights of stairs to reach his flat. The flat itself is meagerly furnished, and the building is located in London, the chief city of Airstrip One.')

### 6.9. Map Reduce LCEL Chain

In [6]:

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)
map_doc_chain = map_doc_prompt | llm


def map_docs(inputs) -> str:
    """Return a list of documents relevant to the question."""
    documents = inputs['documents']
    question = inputs['question']
    results = "\n\n".join(map_doc_chain.invoke({
            "context": doc.page_content,
            "question": question
        }).content for doc in documents)
    return results

map_chain = { 
        "documents": retriever,  # List[Documents]
        "question": RunnablePassthrough()  # "Describe Victory Mansions"
    } | RunnableLambda(map_docs)  # -> a long string of answers concatenated from each document

final_prompt= ChatPromptTemplate.from_messages([
    ("system",
        """
        Given the following extracted parts of a long document and a question, create a final answer. 
        If you don't know the answer, just say that you don't know. Don't try to make up an answer.
        ------
        {context}
        """
    ),
    ("human", "{question}")
])

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm
# chain.invoke("Describe Victory Mansions")
chain.invoke("Where does Winston go to work?")

AIMessage(content='Winston goes to work at the Ministry of Truth.')